# Bootstrapping
## A08 
**Isabela Torres Septien Uribe**  
730667  


In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.utils import resample


## Modelo 1

In [50]:
df = pd.read_csv("Advertising.csv")
df

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9
...,...,...,...,...,...
195,196,38.2,3.7,13.8,7.6
196,197,94.2,4.9,8.1,9.7
197,198,177.0,9.3,6.4,12.8
198,199,283.6,42.0,66.2,25.5


In [51]:

X = df["TV"].values.reshape(-1,1)
y = df["sales"].values.reshape(-1,1)

In [ ]:
lr = LinearRegression()
lr.fit(X,y)

b0 = lr.intercept_
b1 = lr.coef_

print(f"Intercept (b0): {b0[0]}")
print(f"Slope (b1): {b1[0][0]}")

Intercept (b0): 7.032593549127695
Slope (b1): 0.04753664043301975


In [53]:
x_bar = np.mean(X)
y_bar = np.mean(y)
n = len(X)

y_hat = lr.predict(X)


In [54]:
RSS = np.sum((y_hat - y)**2)

sigma_cuad = RSS/(n-2)

beta_1_m1 = np.sqrt(sigma_cuad/(((X - x_bar)**2).sum()))

beta_0_m1 = np.sqrt(sigma_cuad * (1/n + (x_bar**2/(((X - x_bar)**2).sum()))))

print(f"Standard error of b1: {beta_1_m1}")
print(f"Standard error of b0: {beta_0_m1}")

Standard error of b1: 0.0026906071877968707
Standard error of b0: 0.4578429402734785


In [63]:
# hacemos la regression lineal con ridge 
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=1.0)
ridge.fit(X, y)
B0R = ridge.coef_
B1R = ridge.intercept_
print(f"Intercept (b0)", B1R)
print(f"Slope (b1)", B0R)

Intercept (b0) [7.03259831]
Slope (b1) [0.04753661]


In [66]:
x_bar = np.mean(X)
y_bar = np.mean(y)
n = len(X)

y_hat = lr.predict(X)

In [67]:
RSS = np.sum((y_hat - y)**2)

sigma_cuad = RSS/(n-2)

beta_1_m1R = np.sqrt(sigma_cuad/(((X - x_bar)**2).sum()))

beta_0_m1R = np.sqrt(sigma_cuad * (1/n + (x_bar**2/(((X - x_bar)**2).sum()))))

print(f"Standard error of b1: {beta_1_m1R}")
print(f"Standard error of b0: {beta_0_m1R}")

Standard error of b1: 0.0026906071877968707
Standard error of b0: 0.4578429402734785


## Modelo 2

In [55]:
df = pd.read_csv("Default.csv")


In [56]:
df['default']  = df['default'].astype('category')
df['student']  = df['student'].astype('category')

In [57]:
y2 = df['default'] == 'Yes'
X2 = df['balance']


In [58]:
model = LogisticRegression()
model.fit(X2.values.reshape(-1,1), y2)
B0 = model.intercept_
B1 = model.coef_
B0, B1

(array([-10.65132824]), array([[0.00549892]]))

In [60]:
x_const = sm.add_constant(X2)
model = sm.Logit(y2, x_const).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.079823
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9998
Method:                           MLE   Df Model:                            1
Date:              jue., 20 nov. 2025   Pseudo R-squ.:                  0.4534
Time:                        17:17:18   Log-Likelihood:                -798.23
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                6.233e-290
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -10.6513      0.361    -29.491      0.000     -11.359      -9.943
balance        0.0055      0.000     24.952      0.000       0.005       0.006
==============================================================================

Possibly complete quasi-separation: A fraction 0.13 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

## Bootstrap

In [ ]:
#resample con mi modelo 1
B0_bootstrap_m1 = []
B1_bootstrap_m1 = []

for i in range(1000):
    X_resampled, y_resampled = resample(X, y)
    lr_bootstrap = LinearRegression()
    lr_bootstrap.fit(X_resampled, y_resampled)
    B0_bootstrap_m1.append(lr_bootstrap.intercept_)
    B1_bootstrap_m1.append(lr_bootstrap.coef_)

In [ ]:
B0_bootstrap_m1 = np.array(B0_bootstrap_m1)
B1_bootstrap_m1 = np.array(B1_bootstrap_m1).flatten()

print("Media B0 bootstrap:", B0_bootstrap_m1.mean())
print("SE B0 bootstrap:", B0_bootstrap_m1.std())

print("Media B1 bootstrap:", B1_bootstrap_m1.mean())
print("SE B1 bootstrap:", B1_bootstrap_m1.std())

Media B0 bootstrap: 7.042208312128683
SE B0 bootstrap: 0.33268521613670754
Media B1 bootstrap: 0.047471236983978385
SE B1 bootstrap: 0.002855397480250594


In [64]:
#resample con mi modelo 1
B0_bootstrap_m1R = []
B1_bootstrap_m1R = []

for i in range(1000):
    X_resampled, y_resampled = resample(X, y)
    R_bootstrap = Ridge(alpha=1.0)
    R_bootstrap.fit(X_resampled, y_resampled)
    B0_bootstrap_m1R.append(R_bootstrap.intercept_)
    B1_bootstrap_m1R.append(R_bootstrap.coef_)

In [65]:
B0_bootstrap_m1R = np.array(B0_bootstrap_m1R)
B1_bootstrap_m1R = np.array(B1_bootstrap_m1R).flatten()

print("Media B0 bootstrap Ridge:", B0_bootstrap_m1R.mean())
print("SE B0 bootstrap Ridge:", B0_bootstrap_m1R.std())

print("Media B1 bootstrap Ridge:", B1_bootstrap_m1R.mean())
print("SE B1 bootstrap Ridge:", B1_bootstrap_m1R.std())

Media B0 bootstrap Ridge: 7.02979413630033
SE B0 bootstrap Ridge: 0.3251249094690748
Media B1 bootstrap Ridge: 0.04757009747805836
SE B1 bootstrap Ridge: 0.002771086174042682


In [ ]:
#resample con mi modelo 2

B0_bootstrap_m2 = []
B1_bootstrap_m2 = []
for i in range(1000):
    X2_resampled, y2_resampled = resample(X2, y2)
    lreg_bootstrap = LogisticRegression()
    lreg_bootstrap.fit(X2_resampled.values.reshape(-1,1), y2_resampled)
    B0_bootstrap_m2.append(lreg_bootstrap.intercept_)
    B1_bootstrap_m2.append(lreg_bootstrap.coef_)

In [ ]:

B0_bootstrap_m2 = np.array(B0_bootstrap_m2)
B1_bootstrap_m2 = np.array(B1_bootstrap_m2).flatten()

print("Media B0 bootstrap:", B0_bootstrap_m2.mean())
print("SE B0 bootstrap:", B0_bootstrap_m2.std())

print("Media B1 bootstrap:", B1_bootstrap_m2.mean())
print("SE B1 bootstrap:", B1_bootstrap_m2.std())

Media B0 bootstrap: -10.683372029063339
SE B0 bootstrap: 0.3516588750773237
Media B1 bootstrap: 0.005518037507720946
SE B1 bootstrap: 0.00021384400261518715


In [73]:
#Tabla comparativa

tabla = pd.DataFrame({
    "Modelo": ["Regresión Lineal", "Regresión Logística", "Regresión Lineal con Bootstrap", "Regresión Logística con Bootstrap", "Regresión Lineal Ridge","Regresión Lineal Ridge con Bootstrap"],
    "B0 Media": [b0, B0, B0_bootstrap_m1.mean(), B0_bootstrap_m2.mean(),B1R ,B0_bootstrap_m1R.mean()],
    "B0 SE": [beta_0_m1, 0.361, B0_bootstrap_m1.std(), B0_bootstrap_m2.std(),beta_0_m1R , B0_bootstrap_m1R.std()],
    "B1 Media": [b1, B1, B1_bootstrap_m1.mean(), B1_bootstrap_m2.mean(),B0R, B1_bootstrap_m1R.mean()],
    "B1 SE": [beta_1_m1, 0.000, B1_bootstrap_m1.std(), B1_bootstrap_m2.std(), beta_1_m1R,B1_bootstrap_m1R.std()]})
tabla


,Modelo,B0 Media,B0 SE,B1 Media,B1 SE
0,Regresión Lineal,[7.032593549127695],0.457843,[[0.04753664043301975]],0.002691
1,Regresión Logística,[-10.651328236144845],0.361000,[[0.005498915546890768]],0.000000
2,Regresión Lineal con Bootstrap,7.042208,0.332685,0.047471,0.002855
3,Regresión Logística con Bootstrap,-10.683372,0.351659,0.005518,0.000214
4,Regresión Lineal Ridge,[7.032598314474717],0.457843,[0.04753660802506272],0.002691
5,Regresión Lineal Ridge con Bootstrap,7.029794,0.325125,0.04757,0.002771


## Conclusiones

En el siguiente ejercicio tomamos en cuenta los modelos de Regresión Logistica y Reggresión Lineal para analizar mis betas. De esta forma, pudimos medir la variabilidad de nuestros coedicientes y comprobarlo con nuestras regresiones, usamos 1000 simulaciones de Bootsrap. En la primera parte de Nuesto Modelo 1, podemos ver que todas nuestras metricas coinciden de la Regresión Lineal con la Regresión Lineal con Bootstrap a excepción de los errores estandar de B0, a pesar de esto, podemos decir con confianza que los supuestos de nuestro modelo si coinciden y las estimaciones son confiables. 

De igual manera, en el modelo 2 ahí podemos ver que tenemos menor desviación en los resultados y que en todos nuestros resultados se cumplen los supuestos del modelo logistico y que las estumación son correctas para el dataset. Finalmente realizamos una regressión lineal con ridge en el bootstrap, por lo que podemos confirmar nuestros resultados, esta ves con niestro modelo Bootstrap tenemos todavia menor diferencia.